# Using autodiff to check gradient/Hessians

In [10]:
using Revise
using DataFrames, Random, GLM, QuasiCopula
using ForwardDiff, Test, LinearAlgebra
using LinearAlgebra: BlasReal, copytri!
using ToeplitzMatrices
using BenchmarkTools
using SnpArrays
using ForwardDiff
# using MendelPlots
ENV["COLUMNS"] = 240


BLAS.set_num_threads(1)
Threads.nthreads()

function simulate_random_snparray(s::Union{String, UndefInitializer}, n::Int64,
    p::Int64; mafs::Vector{Float64}=zeros(Float64, p), min_ma::Int = 5)

    #first simulate a random {0, 1, 2} matrix with each SNP drawn from Binomial(2, r[i])
    A1 = BitArray(undef, n, p) 
    A2 = BitArray(undef, n, p) 
    for j in 1:p
        minor_alleles = 0
        maf = 0
        while minor_alleles <= min_ma
            maf = 0.5rand()
            for i in 1:n
                A1[i, j] = rand(Bernoulli(maf))
                A2[i, j] = rand(Bernoulli(maf))
            end
            minor_alleles = sum(view(A1, :, j)) + sum(view(A2, :, j))
        end
        mafs[j] = maf
    end

    #fill the SnpArray with the corresponding x_tmp entry
    return _make_snparray(s, A1, A2)
end

function _make_snparray(s::Union{String, UndefInitializer}, A1::BitArray, A2::BitArray)
    n, p = size(A1)
    x = SnpArray(s, n, p)
    for i in 1:(n*p)
        c = A1[i] + A2[i]
        if c == 0
            x[i] = 0x00
        elseif c == 1
            x[i] = 0x02
        elseif c == 2
            x[i] = 0x03
        else
            throw(MissingException("matrix shouldn't have missing values!"))
        end
    end
    return x
end

┌ Warning: /Users/biona001/.julia/packages/AbstractFFTs/4iQz5/src/chainrules.jl no longer exists, deleted all methods
└ @ Revise ~/.julia/packages/Revise/7HQ7u/src/packagedef.jl:667
┌ Error: Failed to revise /Users/biona001/.julia/dev/QuasiCopula/src/gwas/multivariate.jl
│   exception =
│    invalid redefinition of constant MultivariateCopulaVCObs
│    Stacktrace:
│     [1] top-level scope
│       @ ~/.julia/dev/QuasiCopula/src/gwas/multivariate.jl:2
│    Revise evaluation error at /Users/biona001/.julia/dev/QuasiCopula/src/gwas/multivariate.jl:2
│    
└ @ Revise ~/.julia/packages/Revise/7HQ7u/src/packagedef.jl:723
┌ Error: Failed to revise /Users/biona001/.julia/dev/QuasiCopula/src/gwas/multivariate_gwas_autodiff.jl
│   exception =
│    LoadError: UndefVarError: @showprogress not defined
│    in expression starting at /Users/biona001/.julia/dev/QuasiCopula/src/gwas/multivariate_gwas_autodiff.jl:32
└ @ Revise ~/.julia/packages/Revise/7HQ7u/src/packagedef.jl:725
┌ Error: Failed to revis

_make_snparray (generic function with 1 method)

## simulate data

In [11]:
# simulate data
p = 5    # number of fixed effects, including intercept
m = 2    # number of variance componentsac
n = 1000 # number of sample
d = 3    # number of phenotypes per sample
q = 1000 # number of SNPs
k = 0   # number of causal SNPs

# sample d marginal distributions for each phenotype within samples
Random.seed!(2022)
possible_distributions = [Bernoulli, Poisson, Normal]
vecdist = rand(possible_distributions, d)
veclink = [canonicallink(vecdist[j]()) for j in 1:d]

# simulate nongenetic coefficient and variance component params
Random.seed!(2022)
τtrue = 10.0
Btrue = rand(Uniform(-0.5, 0.5), p, d)
θtrue = [0.1, 0.1]
V1 = ones(d, d)
V2 = Matrix(I, d, d)
Γ = θtrue[1] * V1 + θtrue[2] * V2

# simulate non-genetic design matrix
Random.seed!(2022)
X = [ones(n) randn(n, p - 1)]

# simulate random SnpArray with q SNPs and randomly choose k SNPs to be causal
Random.seed!(2022)
G = simulate_random_snparray(undef, n, q)
Gfloat = convert(Matrix{Float64}, G, center=true, scale=true)
γtrue = zeros(q, d)
causal_snps = sample(1:q, k, replace=false) |> sort
for j in 1:d
    γtrue[causal_snps, j] .= rand([-1, 1], k)
end

# sample phenotypes
Y = zeros(n, d)
y = Vector{Float64}(undef, d)
for i in 1:n
    Xi = X[i, :]
    Gi = Gfloat[i, :]
    η = Btrue' * Xi + γtrue' * Gi
    vecd_tmp = Vector{UnivariateDistribution}(undef, d)
    for j in 1:d
        dist = vecdist[j]
        μj = GLM.linkinv(canonicallink(dist()), η[j])
        if dist == Normal
            σ2 = inv(τtrue)
            σ = sqrt(σ2)
            vecd_tmp[j] = Normal(μj, σ)
        else
            vecd_tmp[j] = dist(μj)
        end
    end
    multivariate_dist = MultivariateMix(vecd_tmp, Γ)
    res = Vector{Float64}(undef, d)
    rand(multivariate_dist, y, res)
    Y[i, :] .= y
end

# form model
V = m == 1 ? [V1] : [V1, V2]
qc_model = MultivariateCopulaVCModel(Y, X, V, vecdist, veclink)
# initialize_model!(qc_model)

[qc_model.Y[5, :] vecdist]

3×2 Matrix{Any}:
  1.0       Bernoulli
 -0.838267  Normal
  0.0       Poisson

## Is $\nabla_\beta res$ calculated correctly? 

We can check using ForwardDiff

The function is 

$$res_{ij}(\beta) = \frac{y_{ij} - \mu_{ij}}{\sqrt{\sigma_{ij}^2(\beta)}}$$

### Normal

Assumes y, X are given. We calculate the residuals for just 1 sample

In [4]:
# sample data
xi = qc_model.X[1, :] # p by 1
yi = qc_model.Y[1, :] # d by 1

# objective
function resβ(y, x, vecB::AbstractVector)
    p = length(x)
    d = length(y)
    B = reshape(vecB, p, d)
    η = B' * x
    μ = GLM.linkinv.(IdentityLink(), η)
    varμ = GLM.glmvar.(Normal(), μ)
    return (y - μ) ./ sqrt.(varμ)
end
resβ(B::AbstractMatrix) = resβ(yi, xi, vec(B))
resβ(B::AbstractVector) = resβ(yi, xi, B)

B = randn(p, d)
@show resβ(yi, xi, vec(B))

# autodiff gradient
∇resβ_autodiff = x -> ForwardDiff.jacobian(resβ, x)
correct = ∇resβ_autodiff(vec(B))'

# mathematical gradient
function ∇resβ(x, y, vecB::AbstractVector{T}) where T
    p = length(x)
    d = length(y)
    B = reshape(vecB, p, d)
    η = B' * x
    μ = GLM.linkinv.(IdentityLink(), η)
    varμ = GLM.glmvar.(Normal(), μ)
    res = y - μ
    std_res = res ./ sqrt.(varμ)
    dμ = GLM.mueta.(IdentityLink(), η)
    out = zeros(T, d*p, d)
    @inbounds for j in 1:d
        for k in 1:p
            out[(j-1)*p + k, j] = QuasiCopula.update_∇res_ij(Normal(), x[k], 
                std_res[j], μ[j], dμ[j], varμ[j])
        end
    end
    return out
end
math_result = ∇resβ(xi, yi, vec(B))

# compare results
[vec(math_result) vec(correct)]

resβ(yi, xi, vec(B)) = [-2.6474612012464025, 1.5524540653281953, 0.10154420769843686]


45×2 Matrix{Float64}:
 -1.0       -1.0
  0.308648   0.308648
 -2.26841   -2.26841
 -0.316064  -0.316064
 -0.66411   -0.66411
  0.0        0.0
  0.0        0.0
  0.0        0.0
  0.0        0.0
  0.0        0.0
  0.0        0.0
  0.0        0.0
  0.0        0.0
  ⋮         
  0.0       -0.0
  0.0       -0.0
  0.0       -0.0
  0.0       -0.0
  0.0       -0.0
  0.0       -0.0
  0.0       -0.0
 -1.0       -1.0
  0.308648   0.308648
 -2.26841   -2.26841
 -0.316064  -0.316064
 -0.66411   -0.66411

### Bernoulli

In [6]:
# sample data
xi = qc_model.X[1, :] # p by 1
yi = qc_model.Y[1, :] # d by 1

# objective
function resβ(y, x, vecB::AbstractVector)
    p = length(x)
    d = length(y)
    B = reshape(vecB, p, d)
    η = B' * x
    μ = GLM.linkinv.(LogitLink(), η)
    varμ = GLM.glmvar.(Bernoulli(), μ)
    return (y - μ) ./ sqrt.(varμ)
end
resβ(B::AbstractMatrix) = resβ(yi, xi, vec(B))
resβ(B::AbstractVector) = resβ(yi, xi, B)

B = randn(p, d)
@show resβ(yi, xi, vec(B))

# autodiff gradient
∇resβ_autodiff = x -> ForwardDiff.jacobian(resβ, x)
correct = ∇resβ_autodiff(vec(B))'

# mathematical gradient
function ∇resβ(x, y, vecB::AbstractVector{T}) where T
    p = length(x)
    d = length(y)
    B = reshape(vecB, p, d)
    η = B' * x
    μ = GLM.linkinv.(LogitLink(), η)
    varμ = GLM.glmvar.(Bernoulli(), μ)
    res = y - μ
    std_res = res ./ sqrt.(varμ)
    dμ = GLM.mueta.(LogitLink(), η)
    out = zeros(T, d*p, d)
    @inbounds for j in 1:d
        for k in 1:p
            out[(j-1)*p + k, j] = QuasiCopula.update_∇res_ij(Bernoulli(), x[k], 
                std_res[j], μ[j], dμ[j], varμ[j])
        end
    end
    return out
end
math_result = ∇resβ(xi, yi, vec(B))

# compare results
[vec(math_result) vec(correct)]

resβ(yi, xi, vec(B)) = [-1.699553635820773, -2.9658266436544203, 2.621786668608018]


45×2 Matrix{Float64}:
 -0.849777  -0.849777
  0.262282   0.262282
 -1.92764   -1.92764
 -0.268584  -0.268584
 -0.564345  -0.564345
  0.0        0.0
  0.0        0.0
  0.0        0.0
  0.0        0.0
  0.0        0.0
  0.0        0.0
  0.0        0.0
  0.0        0.0
  ⋮         
  0.0       -0.0
  0.0       -0.0
  0.0       -0.0
  0.0       -0.0
  0.0       -0.0
  0.0       -0.0
  0.0       -0.0
 -1.31089   -1.31089
  0.404604   0.404604
 -2.97365   -2.97365
 -0.414327  -0.414327
 -0.870577  -0.870577

### Poisson

In [7]:
# sample data
xi = qc_model.X[1, :] # p by 1
yi = qc_model.Y[1, :] # d by 1

# objective
function resβ(y, x, vecB::AbstractVector)
    p = length(x)
    d = length(y)
    B = reshape(vecB, p, d)
    η = B' * x
    μ = GLM.linkinv.(LogLink(), η)
    varμ = GLM.glmvar.(Poisson(), μ)
    return (y - μ) ./ sqrt.(varμ)
end
resβ(B::AbstractMatrix) = resβ(yi, xi, vec(B))
resβ(B::AbstractVector) = resβ(yi, xi, B)

B = randn(p, d)
@show resβ(yi, xi, vec(B))

# autodiff gradient
∇resβ_autodiff = x -> ForwardDiff.jacobian(resβ, x)
correct = ∇resβ_autodiff(vec(B))'

# mathematical gradient
function ∇resβ(x, y, vecB::AbstractVector{T}) where T
    p = length(x)
    d = length(y)
    B = reshape(vecB, p, d)
    η = B' * x
    μ = GLM.linkinv.(LogLink(), η)
    varμ = GLM.glmvar.(Poisson(), μ)
    res = y - μ
    std_res = res ./ sqrt.(varμ)
    dμ = GLM.mueta.(LogLink(), η)
    out = zeros(T, d*p, d)
    @inbounds for j in 1:d
        for k in 1:p
            out[(j-1)*p + k, j] = QuasiCopula.update_∇res_ij(Poisson(), x[k], 
                std_res[j], μ[j], dμ[j], varμ[j])
        end
    end
    return out
end
math_result = ∇resβ(xi, yi, vec(B))

# compare results
[vec(math_result) vec(correct)]

resβ(yi, xi, vec(B)) = [-0.6311261621867231, -2.784614111314093, 18.689464943875688]


45×2 Matrix{Float64}:
  -0.315563    -0.315563
   0.0973979    0.0973979
  -0.715827    -0.715827
  -0.0997383   -0.0997383
  -0.209568    -0.209568
   0.0          0.0
   0.0          0.0
   0.0          0.0
   0.0          0.0
   0.0          0.0
   0.0          0.0
   0.0          0.0
   0.0          0.0
   ⋮          
   0.0         -0.0
   0.0         -0.0
   0.0         -0.0
   0.0         -0.0
   0.0         -0.0
   0.0         -0.0
   0.0         -0.0
  -9.39809     -9.39809
   2.9007       2.9007
 -21.3187     -21.3187
  -2.9704      -2.9704
  -6.24136     -6.24136

## Check $\nabla_{\beta} L$

In [12]:
# sample data
i = 10
xi = qc_model.X[i, :] # p by 1
yi = qc_model.Y[i, :] # d by 1

function A_mul_b!(c::AbstractVector{T}, A::AbstractMatrix, b::AbstractVector) where T
    n, p = size(A)
    fill!(c, zero(T))
    for j in 1:p, i in 1:n
        c[i] += A[i, j] * b[j]
    end
    return c
end

# loglikelihood for a single sample
function loglikelihood(y, x, vecB::AbstractVector{T}, qc_model) where T
    p = length(x)
    d = length(y)
    m = qc_model.m
    B = reshape(vecB, p, d)
    η = B' * x
    μ = GLM.linkinv.(qc_model.veclink, η)
    varμ = GLM.glmvar.(qc_model.vecdist, μ)
    res = y - μ
    std_res = res ./ sqrt.(varμ)
    dμ = GLM.mueta.(LogLink(), η)
    storage_d = zeros(T, d)
    q = zeros(T, m)
    # GLM loglikelihood (term 2)
    logl = zero(T)
    @inbounds for j in eachindex(y)
        logl += QuasiCopula.loglik_obs(qc_model.vecdist[j], y[j], μ[j], one(T), one(T))
    end
    # loglikelihood term 1 i.e. -sum ln(1 + 0.5tr(Γ(θ)))
    tsum = dot(qc_model.θ, qc_model.t) # tsum = 0.5tr(Γ)
    logl += -log(1 + tsum)
    # loglikelihood term 3 i.e. sum ln(1 + 0.5 r*Γ*r)
    @inbounds for k in 1:qc_model.m # loop over m variance components
        mul!(storage_d, qc_model.V[k], std_res) # storage_d = V[k] * r
#         if needgrad
#             BLAS.gemv!('N', θ[k], qc.∇resβ, qc.storage_d, one(T), qc.∇vecB) # ∇β = ∇r*Γ*r
#         end
        q[k] = dot(std_res, storage_d) / 2 # q[k] = 0.5 r * V[k] * r
    end
    qsum = dot(qc_model.θ, q) # qsum = 0.5 r*Γ*r
    logl += log(1 + qsum)
    return logl
end
loglikelihood(B::AbstractMatrix) = loglikelihood(yi, xi, vec(B), qc_model)
loglikelihood(B::AbstractVector) = loglikelihood(yi, xi, B, qc_model)

# autodiff gradient
logl_autodiff = x -> ForwardDiff.gradient(loglikelihood, x)
correct = logl_autodiff(vec(B))

# gradient from math
qc_model.B .= B
loglikelihood!(qc_model, true, false)

[correct qc_model.data[i].∇vecB]

15×2 Matrix{Float64}:
  0.292198    0.292198
  0.201006    0.201006
  0.194317    0.194317
  0.0109756   0.0109756
  0.349726    0.349726
 -0.674226   -0.674226
 -0.463807   -0.463807
 -0.448371   -0.448371
 -0.0253255  -0.0253255
 -0.806966   -0.806966
  4.75121     4.75121
  3.2684      3.2684
  3.15963     3.15963
  0.178466    0.178466
  5.68662     5.68662

## Check $\nabla_{\theta} L$

In [13]:
# sample data
i = 5
xi = qc_model.X[i, :] # p by 1
yi = qc_model.Y[i, :] # d by 1

function A_mul_b!(c::AbstractVector{T}, A::AbstractMatrix, b::AbstractVector) where T
    n, p = size(A)
    fill!(c, zero(T))
    for j in 1:p, i in 1:n
        c[i] += A[i, j] * b[j]
    end
    return c
end

# loglikelihood for a single sample
function loglikelihood(y, x, θ::AbstractVector{T}, qc_model) where T
    p = length(x)
    d = length(y)
    m = qc_model.m
    B = qc_model.B
    η = B' * x
    μ = GLM.linkinv.(qc_model.veclink, η)
    varμ = GLM.glmvar.(qc_model.vecdist, μ)
    res = y - μ
    std_res = res ./ sqrt.(varμ)
    storage_d = zeros(T, d)
    q = zeros(T, m)
    # GLM loglikelihood (term 2)
    logl = zero(T)
    @inbounds for j in eachindex(y)
        logl += QuasiCopula.loglik_obs(qc_model.vecdist[j], y[j], μ[j], one(T), one(T))
    end
    # loglikelihood term 1 i.e. -sum ln(1 + 0.5tr(Γ(θ)))
    tsum = dot(θ, qc_model.t) # tsum = 0.5tr(Γ)
    logl += -log(1 + tsum)
    # loglikelihood term 3 i.e. sum ln(1 + 0.5 r*Γ*r)
    @inbounds for k in 1:qc_model.m # loop over m variance components
        mul!(storage_d, qc_model.V[k], std_res) # storage_d = V[k] * r
#         if needgrad
#             BLAS.gemv!('N', θ[k], qc.∇resβ, qc.storage_d, one(T), qc.∇vecB) # ∇β = ∇r*Γ*r
#         end
        q[k] = dot(std_res, storage_d) / 2 # q[k] = 0.5 r * V[k] * r
    end
    qsum = dot(θ, q) # qsum = 0.5 r*Γ*r
    logl += log(1 + qsum)
    return logl
end
loglikelihood(θ::AbstractVector) = loglikelihood(yi, xi, θ, qc_model)

# autodiff gradient
logl_autodiff = x -> ForwardDiff.gradient(loglikelihood, x)

θ = rand(2)
correct = logl_autodiff(θ)

# gradient from math
qc_model.θ .= θ
loglikelihood!(qc_model, true, false)

[correct qc_model.data[i].∇θ]

2×2 Matrix{Float64}:
 -0.60808   -0.60808
 -0.496524  -0.496524

## Check $\nabla_{\phi} L$

In [ ]:
# sample data
i = 5
xi = qc_model.X[i, :] # p by 1
yi = qc_model.Y[i, :] # d by 1

function A_mul_b!(c::AbstractVector{T}, A::AbstractMatrix, b::AbstractVector) where T
    n, p = size(A)
    fill!(c, zero(T))
    for j in 1:p, i in 1:n
        c[i] += A[i, j] * b[j]
    end
    return c
end

# loglikelihood for a single sample
function loglikelihood(y, x, ϕ::AbstractVector{T}, qc_model) where T
    p = length(x)
    d = length(y)
    m = qc_model.m
    B = qc_model.B
    η = B' * x
    μ = GLM.linkinv.(qc_model.veclink, η)
    varμ = GLM.glmvar.(qc_model.vecdist, μ)
    res = y - μ
    std_res = T[]
    nuisance_counter = 1
    for j in eachindex(res)
        if typeof(qc_model.vecdist[j]) <: Normal
            τ = abs(qc_model.ϕ[nuisance_counter])
            push!(std_re, res * sqrt(τ))
            nuisance_counter += 1
        else
            push!(std_res, res / sqrt(varμ[j]))
        end
    end
#     std_res = res ./ sqrt.(varμ)
    storage_d = zeros(T, d)
    q = zeros(T, m)
    # GLM loglikelihood (term 2)
    logl = zero(T)
    @inbounds for j in eachindex(y)
        logl += QuasiCopula.loglik_obs(qc_model.vecdist[j], y[j], μ[j], one(T), one(T))
    end
    # loglikelihood term 1 i.e. -sum ln(1 + 0.5tr(Γ(θ)))
    tsum = dot(θ, qc_model.t) # tsum = 0.5tr(Γ)
    logl += -log(1 + tsum)
    # also add contributions from nuisance parameters
    for (j, idx) in enumerate(qc_model.nuisance_idx)
        τ = abs(qc_model.ϕ[j])
        rss = abs2(norm(qc.std_res[idx]))
        logl += -(1 * log(2π) - 1 * log(abs(τ)) + rss) / 2
    end
    # loglikelihood term 3 i.e. sum ln(1 + 0.5 r*Γ*r)
    @inbounds for k in 1:qc_model.m # loop over m variance components
        mul!(storage_d, qc_model.V[k], std_res) # storage_d = V[k] * r
#         if needgrad
#             BLAS.gemv!('N', θ[k], qc.∇resβ, qc.storage_d, one(T), qc.∇vecB) # ∇β = ∇r*Γ*r
#         end
        q[k] = dot(std_res, storage_d) / 2 # q[k] = 0.5 r * V[k] * r
    end
    qsum = dot(θ, q) # qsum = 0.5 r*Γ*r
    logl += log(1 + qsum)
    return logl
end
loglikelihood(ϕ::AbstractVector) = loglikelihood(yi, xi, ϕ, qc_model)

# autodiff gradient
logl_autodiff = x -> ForwardDiff.gradient(loglikelihood, x)

θ = rand(2)
correct = logl_autodiff(θ)

# gradient from math
qc_model.θ .= θ
loglikelihood!(qc_model, true, false)

[correct qc_model.data[i].∇θ]